In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

# in those notebook we're going to cover some of the most fundamental concepts of tensorflow

more specifically we're going to cover:
* Intro to tensors
* getting info from tensors
* manipulating tensors
* tensors and numpy
* using @tf.function (a way to speed up your regular python functions
* using gpu with tf
* exercises to try yourself

# Intro to tensors

In [2]:
#import tensorflow
import tensorflow as tf
print(tf.__version__)


2.4.1


### creating tensors with tf.constant()

In [17]:
#create first tensors with tf.constant()

scalar = tf.constant(7)

scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
#check ndim of tensor
scalar.ndim

0

In [5]:
#create a vector
vector = tf.constant([10,10])
print(vector)

tf.Tensor([10 10], shape=(2,), dtype=int32)


In [6]:
#check to ndim of vector
vector.ndim

1

In [7]:
#create a matrix
matrix = tf.constant([[10,7],[7,10]])
print(matrix)

tf.Tensor(
[[10  7]
 [ 7 10]], shape=(2, 2), dtype=int32)


In [8]:
#get matrix ndim
matrix.ndim
#the number of elements in a tensors shape represents the number of dimensions

2

In [9]:
#create another matrix with specified dtype
another_matrix = tf.constant([[10.,7.],
                             [3.,2.],
                            [8.,9.]], dtype = tf.float16)
print(another_matrix)
#by default, tensors are created as int32

tf.Tensor(
[[10.  7.]
 [ 3.  2.]
 [ 8.  9.]], shape=(3, 2), dtype=float16)


In [10]:
#what is ndim of another matrix?
another_matrix.ndim

2

In [11]:
#now let's create a tensor
tensor = tf.constant([[[1,2,3],
                      [4,5,6]],
                     [[7,8,9],
                     [10,11,12]],
                     [[13,14,15],
                     [16,17,18]]])
print(tensor)

tf.Tensor(
[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]

 [[13 14 15]
  [16 17 18]]], shape=(3, 2, 3), dtype=int32)


In [12]:
#find tensor number of dimensions
tensor.ndim

3

What we've created so far:
* scalar: a single number
* vector: a number with direction (e.g. wind speed and direction)
* matrix: a 2-d array of numbers
* tensor: an n-dimension array of numbers (where n can be any number. a 0-d tensor is scalar, a 1-d tensor is a vector

### create tensors with tf.Variable()

The `Variable()` constructor requires an initial value for the variable, which
can be a `Tensor` of any type and shape. This initial value defines the type
and shape of the variable. After construction, the type and shape of the
variable are fixed. The value can be changed using one of the assign methods.

In [13]:
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor, unchangeable_tensor
#notice output of changeable_tensor is tf.Variable 'Variable:0' shape=(2,)

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [14]:
#let's change the values of changeable_tensor
#changeable_tensor[0] = 7   #this throws an error
#changeable_tensor
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [15]:
#now let's try changing a value of unchangeable_tensor
#unchangeable_tensor[0] = 7   
#unchangeable_tensor[0].assign(7)
#by of these throw errors
#tensorflow.python.framework.ops.EagerTensor' object does not support item assignment
#obviously tf.constant has no .assign() method

rarely in practice will you need to decide whether to use tf.constant or tf.Variable to create tensors, as tf does this for you. however, if in doubt, use tf.constant and chnage it later if needed.

### creating random tensors

* random tensors are tensors of some arbitrary size which contain random numbers.
* random numbers are generally used to intialize weights in neural networks.
* as the nn sees more examples of the data it begins to change these random weights to better predict the outputs

In [21]:
#create 2 random tensors
random1 = tf.random.Generator.from_seed(42)
random1 = random1.normal(shape = (3,2))
random1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [22]:
random2 = tf.random.Generator.from_seed(42)
random2 = random2.uniform(shape = (3,2))
random2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.7493447 , 0.73561966],
       [0.45230794, 0.49039817],
       [0.1889317 , 0.52027524]], dtype=float32)>

### shuffling the order of tensors